In [54]:
import pandas as pd

In [55]:
columName = ['id', 'name']
movies = pd.read_csv('./data/movies.csv',sep=";", names=columName, header=None, index_col=False)
movies_id = movies[['id']].values

In [56]:
rating = pd.read_csv('./data/customers_rating.csv',sep=";")

In [57]:
#função que retorna quantidade de filmes
def getQtdMovie():
    return len(movies)

In [58]:
#função que recebe um dataFrame e id de filme e retorna a média das avaliações.
def getMean(df, movieId):
    count = 0
    summary = 0
    for i in rating[rating.Movie_Id == movieId].values:
        summary += i[1]
        count += 1
    
    return summary/count

In [59]:
#função que recebe o id do filme como parâmetro e retorna nome do filme
def getName(movieId):
    row = movies[movies['id']==movieId][['name']].values.tolist()[0][0]
    return row

In [60]:
#função que cria e retorna uma lista de dicionarios com chaves de id, média e nome.
def getMovieRating():
    arr =[]
    for item in movies_id:
        arr.append({
            "id": item[0], 
            "mean": getMean(rating, item[0]),
            "name":getName(item[0])
            })
        
    return arr

In [61]:
#função que pega a lista de dicionários e ordena a média de forma decrescente e retornando as top 5 médias.
def bestRating():
    arr = getMovieRating()
    order = sorted(arr, key=lambda row:row['mean'], reverse=1)
    best = [order[0], order[1], order[2], order[3], order[4]]
    return best

In [62]:
#função que separa uma coluna do dataFrame em titulo e ano, criado um temp para armazenar o df agrupado pela coluna
#'year' e ordenado de forma crescente.
#retornando uma lista com ano e lançamento.
def getRelease():
    #Quais os 5 anos com menos lançamentos de filmes?
    movies_t=[item.split(', ') for item in movies['name'].values]
    movies['title'] = [item[0].replace('(', '') for item in movies_t]
    movies['year'] = [item[1].replace(')', '') for item in movies_t]
    temp = movies.pivot_table(columns=['year'], aggfunc='size')
    temp = temp.sort_values()
    cont = 0
    temp2 = []
    for i in temp:
        if cont > 4:
            break
        else:
            temp2.append({'year': temp.index[cont], 'release': i})
            cont += 1
    return temp2

In [63]:
#função que pega a data maxima de avaliação, filtra por notas maiores ou iguais a 4.7 e retorna o total de itens.
def getByLastDate():
    lastY = rating.loc[rating['Date'] == rating['Date'].max()]
    lastY = lastY.loc[lastY['Rating'] >= 4.7]
    tot = len(lastY.pivot_table(columns=['Movie_Id'], aggfunc='size'))
    return tot

In [64]:
#função que pega a data maxima de avaliação, agrupa o df por nota e id do filme e retornando os 10 piores filmes.
def getRatingByDate():
    df = rating.loc[rating['Date'] == rating['Date'].max()]
    temp = df.groupby(['Rating','Movie_Id']).agg({'Rating':sum}).head(10).index.tolist()
    temp = [list(item) for item in temp]
    temp = [{"Rating":item[0],"Movie_Id":item[1]} for item in temp]
    return temp

In [65]:
#função que cria um df temporario agrupado pelo id do custumer e ordenado, retornarndo o id do custumer
# e quantas avaliações ele fez.
def getAssessments():   
    #Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?
    temp =rating.pivot_table(columns=['Cust_Id'], aggfunc='size')
    temp = temp.sort_values(ascending=False)
    cont = 0
    temp2 = []
    for i in temp:
        if cont > 4:
            break
        else:
            temp2.append({'Cust_Id': temp.index[cont], 'Assessments': i})
            cont += 1
            
    return temp2

In [66]:
##Quantos filmes estão disponíveis no dataset?
# Um total de 4499 filmes.
print(getQtdMovie())

4499


In [67]:
## Qual é o nome dos 5 filmes com melhor média de avaliação?
# 1° lost: season 1, com média de 4.67
# 2° Ghost in the shell, com média de 4.586
# 3° the simpsons, com média de 4.581
# 4° Inu-Yasha, com média de 4.554
# 5° Lord of de Rings, com média de 4.552
print(bestRating())

[{'id': 3456, 'mean': 4.6709891019450955, 'name': '(Lost: Season 1, 2004)'}, {'id': 3033, 'mean': 4.586363636363636, 'name': '(Ghost in the Shell: Stand Alone Complex: 2nd Gig, 2005)'}, {'id': 2102, 'mean': 4.581295988606693, 'name': '(The Simpsons: Season 6, 1994)'}, {'id': 4238, 'mean': 4.554434413170473, 'name': '(Inu-Yasha, 2000)'}, {'id': 13, 'mean': 4.552, 'name': '(Lord of the Rings: The Return of the King: Extended Edition: Bonus Material, 2003)'}]


In [68]:
##Quais os 5 anos com menos lançamentos de filmes?
# 1° ano 1915 com um lançamento
# 2° ano 1917 com um lançamento
# 3° ano 1922 com um lançamento
# 4° ano 1926 com um lançamento
# 5° ano 1915 com dois lançamentos
print(getRelease())

[{'year': '1915', 'release': 1}, {'year': '1917', 'release': 1}, {'year': '1922', 'release': 1}, {'year': '1926', 'release': 1}, {'year': '1916', 'release': 2}]


In [69]:
##Quantos filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset?
# Com base nos filmes avaliados na última data de avaliação do dataset são 780 filmes.
print(getByLastDate())

780


In [70]:
##Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e quais as notas?
# Todos os dez filmes possuem nota 1, sendo o id do filme 18, 76, 85, 104, 138, 175, 187, 267, 270, 273.  
print(getRatingByDate())

[{'Rating': 1.0, 'Movie_Id': 18}, {'Rating': 1.0, 'Movie_Id': 76}, {'Rating': 1.0, 'Movie_Id': 85}, {'Rating': 1.0, 'Movie_Id': 104}, {'Rating': 1.0, 'Movie_Id': 138}, {'Rating': 1.0, 'Movie_Id': 175}, {'Rating': 1.0, 'Movie_Id': 187}, {'Rating': 1.0, 'Movie_Id': 267}, {'Rating': 1.0, 'Movie_Id': 270}, {'Rating': 1.0, 'Movie_Id': 273}]


In [71]:
##Quais os id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um fez?
# 1° id: 305344, avaliações: 4467
# 2° id: 387418, avaliações: 4422
# 3° id: 2439493, avaliações: 4195
# 4° id: 1664010, avaliações: 4019
# 5° id: 2118461, avaliações: 3769
print(getAssessments())

[{'Cust_Id': 305344, 'Assessments': 4467}, {'Cust_Id': 387418, 'Assessments': 4422}, {'Cust_Id': 2439493, 'Assessments': 4195}, {'Cust_Id': 1664010, 'Assessments': 4019}, {'Cust_Id': 2118461, 'Assessments': 3769}]
